# **Proceso de ETL**

#### En este notebook realizaremos la extracción, transformación y carga (`ETL`) de los conjuntos de datos proporcionados

In [2]:
# Librerias a utilizar
import pandas as pd

---

## En primer lugar, importamos los conjuntos de datos que vamos a utilizar y los observamos.

In [3]:
amazon = pd.read_csv("Datasets/amazon_prime_titles.csv")
amazon.sample() 

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
6593,s6594,Movie,B. A. Pass,Ajay Bahl,"Shilpa Shukla, Shadab Kamal, Rajesh Sharma, Di...",NaN,NaN,2013,18+,99 min,Drama,"Orphaned and at loose ends, a young boy falls ..."


In [4]:
disney = pd.read_csv("Datasets/disney_plus_titles.csv")
disney.sample()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
730,s731,Movie,Apollo: Missions To The Moon,Tom Jennings,"Neil Armstrong, Buzz Aldrin",United States,"November 12, 2019",2019,TV-PG,95 min,"Documentary, Family, Historical","With rare archival footage, this film sheds ne..."


In [5]:
hulu = pd.read_csv("Datasets/hulu_titles.csv")
hulu.sample()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
61,s62,Movie,Maggie's Plan,NaN,NaN,NaN,"October 4, 2021",2015,R,99 min,"Comedy, Drama, Romance","A young woman (Gerwig) set on having a child, ..."


In [6]:
netflix = pd.read_csv("Datasets/netflix_titles.csv")
netflix.sample()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
36,s37,Movie,The Stronghold,Cédric Jimenez,"Gilles Lellouche, Karim Leklou, François Civil...",NaN,"September 17, 2021",2021,TV-MA,105 min,"Action & Adventure, Dramas, International Movies","Tired of the small-time grind, three Marseille..."


---

#### Al observarlos, podemos notar que todos tienen la misma estructura de columnas. Por lo tanto, podemos concatenarlas fácilmente.
#### Pero antes debemos generar la primera transformación que se nos solicita que es particular en cada tabla :
+ Generar campo **`id`**: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = **`as123`**)

In [7]:
# Generamos el nuevo campo "id" concatenando el "show id" + la letra que corresponda
amazon["id"] = "a" + amazon["show_id"] # letra A
disney["id"] = "d" + amazon["show_id"] # letra D
hulu["id"] = "h" + amazon["show_id"] # letra a H
netflix["id"] = "n" + amazon["show_id"] # letra a N

#### Una vez creado el id para cada tabla, procedemos a concatenarlas

In [8]:
# Concatenamos
df_streaming = pd.concat([amazon,disney,hulu,netflix]) 
df_streaming.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",as4
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,as5


In [9]:
# Checkeamos que se haya concatenado correctamente. Por lo que podemos observar se concatenó correctamente 
print("registros totales: ",len(amazon) + len(disney) + len(hulu) + len(netflix))
print("registros totales al concatenar: ",len(df_streaming))

registros totales:  22998
registros totales al concatenar:  22998


In [10]:
# Observamos
df_streaming.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  id            22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.5+ MB


---

#### Ahora procedemos a realizar las siguientes transformaciones solicitadas 

# **`Transformaciones`**: 

+ #### Los valores nulos del campo rating deberán reemplazarse por el string “**`G`**” (corresponde al maturity rating: “general for all audiences”

In [11]:
# Utilizamos el fillna para imputar el string "G"
df_streaming["rating"].fillna("G",inplace=True)

+ #### De haber fechas, deberán tener el formato **`AAAA-mm-dd`**

In [12]:
# Colocamos el formato adecuado de fecha
df_streaming["date_added"] = pd.to_datetime(df_streaming.date_added)

+ #### Los campos de texto deberán estar en **minúsculas**, sin excepciones

In [13]:
#Creacion de una lista con con el nombre de las columnas que no son de tipo numericos para realizar su transformacion a minusculas
columns = df_streaming.columns.drop(['release_year','date_added'])
# Iteramos aplicando el lower de pandas
for i in columns : 
    df_streaming[i] = df_streaming[i].str.lower()

+ #### El campo ***duration*** debe convertirse en dos campos: **`duration_int`** y **`duration_type`**. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [14]:
# Utilizamos el método str.split para separar el campo "duration" a través del espacio 
df_streaming[["duration_int", "duration_type"]] = df_streaming["duration"].str.split(" ", n=1, expand=True)

# Colocamos el tipo de dato correcto para "duration_int"
df_streaming["duration_int"] = df_streaming["duration_int"].astype("Int64")

# Unificamos "seasons" y "season" en la columna duration_type
df_streaming.loc[df_streaming['duration_type']=='seasons', 'duration_type'] = 'season'

---

#### Para las posteriores consultas es necesario tenes la columna "`score`" , la cual se puede obtener promediando los ratings de todos los usuarios del dataset con reseñas para cada película.

In [15]:
# Importamos los data sets de la carpeta "Datasets_ratings", son en total 8 y lo hacemos mediante un ciclo while
i = 1
df1 = pd.read_csv(f"Datasets_ratings/{i}.csv")
i += 1
while i <= 8 :
    df = pd.read_csv(f"Datasets_ratings/{i}.csv")
    df_concat = pd.concat([df1,df])
    df1 = df_concat
    i += 1
df_ratings = df_concat
df_ratings

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500
...,...,...,...,...
1499995,124380,4.5,1196786159,ns5272
1499996,124380,2.5,1196786030,ns5492
1499997,124380,3.5,1196785679,hs305
1499998,124380,4.5,1196787089,ns7881


In [16]:
# Renombramos las columnas
df_ratings.rename(columns = {"movieId" : "id", "rating" : "score"},inplace=True)

# Necesitamos el promedio de cada pelicula
promedio_peliculas = df_ratings.groupby('id')['score'].mean().round(2)

# Renombramos a la columna como score
promedio_peliculas = promedio_peliculas.rename("score").reset_index()

# Observamos
promedio_peliculas

,id,score
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59
...,...,...
22993,ns995,3.52
22994,ns996,3.63
22995,ns997,3.53
22996,ns998,3.58


In [18]:
# Agregar los promedios como una nueva columna en el DataFrame principal
df_streaming = df_streaming.merge(promedio_peliculas, on='id')

# Comprobar que se realizo correctamente
df_streaming[df_streaming["id"] == "ns995"]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,duration_int,duration_type,score
15185,s995,movie,this lady called life,kayode kasum,"bisola aiyeola, efa iwara, molawa onajobi, tin...",nigeria,2021-04-23,2020,tv-14,120 min,"dramas, international movies, romantic movies","abandoned by her family, young single mother a...",ns995,120,min,3.52


In [19]:
# Observamos con .info
df_streaming.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        22998 non-null  object        
 1   type           22998 non-null  object        
 2   title          22998 non-null  object        
 3   director       14739 non-null  object        
 4   cast           17677 non-null  object        
 5   country        11499 non-null  object        
 6   date_added     13444 non-null  datetime64[ns]
 7   release_year   22998 non-null  int64         
 8   rating         22998 non-null  object        
 9   duration       22516 non-null  object        
 10  listed_in      22998 non-null  object        
 11  description    22994 non-null  object        
 12  id             22998 non-null  object        
 13  duration_int   22516 non-null  Int64         
 14  duration_type  22516 non-null  object        
 15  score          2299

In [20]:
# Observamos una muestra
df_streaming.sample(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,duration_int,duration_type,score
1058,s1059,movie,selah and the spades,tayarisha poe,"lovie simone, jharrel jerome, celeste o’connor...",united states,NaT,2020,r,98 min,young adult audience,five factions run the underground life of the ...,as1059,98,min,3.51
3264,s3265,tv show,ancient aliens,NaN,robert clotworthy,NaN,NaT,2014,tv-pg,3 seasons,"documentary, special interest",another season of alien intervention belongs t...,as3265,3,season,3.59
3291,s3292,tv show,america's next top model,NaN,"tyra banks, nigel barker, andré leon talley",NaN,NaT,2010,tv-14,14 seasons,unscripted,america's next top model welcomes one of the m...,as3292,14,season,3.53


In [21]:
# Exportamos el DataFrame con las transformaciones realizadas para su posterior consulta
df_streaming.to_csv('Datasets/df_streaming.csv', index=False)